In [17]:
#import findspark
#findspark.init()
import pyspark
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as func
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from sklearn.model_selection import KFold
import numpy as np

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Assignment 1 Task2") \
    .getOrCreate()

sc = spark.sparkContext

sc.setCheckpointDir('checkpoint/')


In [2]:
lines = spark.read.text("ml-25m/ratings.csv").rdd
parts = lines.map(lambda row: row.value.split(","))

header = parts.first()
parts = parts.filter(lambda line: line != header)

In [3]:
for line in parts.take(5):
    print(line)

['1', '296', '5.0', '1147880044']
['1', '306', '3.5', '1147868817']
['1', '307', '5.0', '1147868828']
['1', '665', '5.0', '1147878820']
['1', '899', '3.5', '1147868510']


In [4]:
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD).cache()

In [5]:
(fold_0, fold_1, fold_2) = ratings.randomSplit([1.0, 1.0, 1.0])

data_list = [fold_0,fold_1,fold_2]
test_list = [fold_2,fold_0,fold_1]

In [6]:
als_1 = ALS(maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
als_2 = ALS(maxIter=25, regParam=0.05, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
als_3 = ALS(maxIter=50, regParam=0.005, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [7]:
rmse_evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
mae_evaluator = RegressionEvaluator(metricName="mae", labelCol="rating",predictionCol="prediction")

In [8]:
def cross_validate(als_version):
    rmse_list =[]
    mae_list =[]
    fold_j = 0
    for fold_i in range(len(data_list)):
        if fold_j < len(data_list)-1:
            fold_j+= 1
        else:
            fold_j = 0
        first_train = data_list[fold_i].union(data_list[fold_j])
        model = als_version.fit(first_train)
        predictions = model.transform(test_list[fold_i])
        
        rmse = rmse_evaluator.evaluate(predictions)
        mae = mae_evaluator.evaluate(predictions)
        
        rmse_list.append(rmse)
        mae_list.append(mae)
    
    return rmse_list, mae_list

In [9]:
rmse_als_1, mae_als_1 = cross_validate(als_1)

In [10]:
rmse_als_2, mae_als_2 = cross_validate(als_2)

In [11]:
rmse_als_3, mae_als_3 = cross_validate(als_3)

In [128]:
import statistics as stat

In [183]:
#for calculate std and mean for rmse metric
als1_rmse_std,als1_rmse_mean = stat.stdev(rmse_als_1),stat.mean(rmse_als_1)
als2_rmse_std,als2_rmse_mean = stat.stdev(rmse_als_2),stat.mean(rmse_als_2)
als3_rmse_std,als3_rmse_mean = stat.stdev(rmse_als_3),stat.mean(rmse_als_3)

#for calculate std and mean for mae metric
als1_mae_std,als1_mae_mean = stat.stdev(mae_als_1),stat.mean(mae_als_1)
als2_mae_std,als2_mae_mean = stat.stdev(mae_als_2),stat.mean(mae_als_2)
als3_mae_std,als3_mae_mean = stat.stdev(mae_als_3),stat.mean(mae_als_3)

In [192]:
rdd_report = sc.parallelize([(rmse_als_1[0],mae_als_1[0],rmse_als_2[0],mae_als_2[0],rmse_als_1[0],mae_als_1[0]),
                            (rmse_als_1[1],mae_als_1[1],rmse_als_2[1],mae_als_2[1],rmse_als_2[1],mae_als_2[1]),
                            (rmse_als_1[2],mae_als_1[2],rmse_als_3[2],mae_als_3[2],rmse_als_3[2],mae_als_3[2]),
                            (als1_rmse_std,als1_mae_std,als2_rmse_std,als2_mae_std,als3_rmse_std,als3_mae_std),
                            (als1_rmse_mean,als1_mae_mean,als2_rmse_mean,als2_mae_mean,als3_rmse_mean,als3_mae_mean)])

In [195]:
df_report = rdd_report.toDF(['ALS Version 1 RMSE Result','ALS Version 1 MAE Result'
                             ,'ALS Version 2 RMSE Result','ALS Version 2 MAE Result'
                            ,'ALS Version 3 RMSE Result','ALS Version 3 MAE Result'])

In [197]:
df_pd = df_report.toPandas()

In [198]:
df_pd.index = ['Fold1','Fold2','Fold3','Std','Mean']

In [199]:
df_pd

,ALS Version 1 RMSE Result,ALS Version 1 MAE Result,ALS Version 2 RMSE Result,ALS Version 2 MAE Result,ALS Version 3 RMSE Result,ALS Version 3 MAE Result
Fold1,0.808220,0.625780,0.789857,0.603656,0.808220,0.625780
Fold2,0.808637,0.626046,0.790064,0.603553,0.790064,0.603553
Fold3,0.808410,0.626003,0.818741,0.612439,0.818741,0.612439
Std,0.000209,0.000143,0.000406,0.000240,0.000493,0.000253
Mean,0.808422,0.625943,0.789734,0.603469,0.819219,0.612544
